In [ ]:
import pandas as pd
dfC = pd.read_csv("non-cliques.csv", delimiter=" ", header=None)
non_cliques = []
for index, row in dfC.iterrows():
    i = 0
    for col in range(0, 12):
        print(row[col])
        try:
            if row[col][0] == "t":
                non_cliques.append(row[col])
        except:
            continue
            

non_cliques

In [32]:

import numpy as np
#Add RiffTrax and Wrestling perhaps

df = pd.read_csv("movie.statistics.csv")
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
cutoffValue = df[df["cluster"]==4].sort_values("cluster.value", ascending=False).head(200)["cluster.value"].values[-1]

def flagClique(row):
    cluster = row["cluster"]
    value = row["cluster.value"]
    title = row["Unnamed: 0"]
    
    if cluster in [3, 5]:
        return 1
    elif title in non_cliques:
        return 0
    elif cluster==4:
        if value < cutoffValue:
            return None
        else:
            return 0
    else:
        return None
df["cliqueFlag"] = df.apply (lambda row: flagClique(row), axis=1)
dfTrain = df.dropna()

dfTrain

,Unnamed: 0,degrees,degress.from.tam,coreness,cluster,cluster.value,cliqueFlag
85,tt0003657,710,3.0,226,1,10.751222,0.0
966,tt0006477,782,3.0,216,1,10.525562,0.0
1008,tt0006580,702,3.0,226,1,10.597453,0.0
1245,tt0007142,1028,3.0,392,1,8.612421,0.0
1583,tt0007823,878,3.0,298,1,11.242820,0.0
...,...,...,...,...,...,...,...
93540,tt9146260,48,4.0,32,5,0.132143,1.0
93891,tt9330502,666,3.0,460,4,0.218198,0.0
94149,tt9486080,302,2.0,196,4,0.235952,0.0
94183,tt9512094,674,3.0,460,4,0.258915,0.0


In [33]:
from sklearn.linear_model import LogisticRegression
dfTrain = dfTrain[~dfTrain.isin([np.nan, np.inf, -np.inf]).any(1)]

clf = LogisticRegression(random_state=0).fit(dfTrain[["degrees", "degress.from.tam", "coreness"]], dfTrain["cliqueFlag"])

In [34]:
print(clf.coef_, clf.intercept_)

[[-0.01514175  1.13952598 -0.03601008]] [6.72657744]


In [35]:
df["predictedClique"] = clf.predict(df[["degrees", "degress.from.tam", "coreness"]])
df

,Unnamed: 0,degrees,degress.from.tam,coreness,cluster,cluster.value,cliqueFlag,predictedClique
0,tt0000009,26,4.0,26,1,0.823608,NaN,1.0
2,tt0001159,146,3.0,146,1,0.900306,NaN,1.0
3,tt0001186,6,3.0,6,1,0.317083,NaN,1.0
4,tt0001230,230,3.0,230,1,0.640627,NaN,0.0
5,tt0001285,208,3.0,166,1,6.123897,NaN,1.0
...,...,...,...,...,...,...,...,...
94877,tt9907824,24,5.0,20,4,0.005730,NaN,1.0
94879,tt9910102,10,5.0,10,4,0.002815,NaN,1.0
94880,tt9910302,52,4.0,38,4,0.020858,NaN,1.0
94881,tt9914522,34,4.0,30,4,0.022572,NaN,1.0


In [37]:
pred_non_clique = df[df["predictedClique"]==0]["Unnamed: 0"].values
pred_non_clique

array(['tt0001230', 'tt0001498', 'tt0002337', ..., 'tt9730696',
       'tt9887154', 'tt9898328'], dtype=object)

In [48]:
pd.DataFrame(pred_non_clique, columns=["pred_non_clique_titles"]).to_csv("pred_non_clique_titles.csv")

In [29]:
len(non_cliques)

400